Here we compare multicriterial selection for counterfactuals with single ones.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time
import pickle
from rf_counterfactuals import RandomForestExplainer, visualize, evaluate_counterfactual, evaluate_counterfactual_set
from rf_counterfactuals.single_cf_costs_functions import heterogeneous_euclidean_overlap_metric, unmatched_components_distance
from rf_counterfactuals.multi_cf_costs_functions import diversity
import os
from collections import defaultdict

from sklearn import preprocessing
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, pairwise_distances
from sklearn.neighbors import NearestNeighbors
DATASET_PATH = "./datasets/"

N_JOBS = -1

# Comment one of two cells below to choose a dataset

In [ ]:
adult_dataset = pd.read_csv(os.path.join(DATASET_PATH, "adult.csv"))

class_feature = "income"
feature_names = [c for c in adult_dataset.columns if c != class_feature]
categorical_features = [1, 3, 5, 6, 7, 8, 9, 13]
to_encode = [c for no, c in enumerate(feature_names) if no in categorical_features]

d = defaultdict(preprocessing.LabelEncoder)

adult_dataset[to_encode] = adult_dataset[to_encode].apply(lambda x: d[x.name].fit_transform(x))

X = adult_dataset.loc[:, adult_dataset.columns!=class_feature]
y = adult_dataset[class_feature]

In [ ]:
# COLUMNS = ['checking_account', 'duration', 'credit_history', 'purpose', 'credit_amount', 'saving_account', 'present_employement',
#           'installment_rate', 'personal_status', 'other_debtors', 'present_residence', 'property', 'age', 'installment_plans',
#           'housing', 'existing_credits_number', 'job', 'people_to_provide_maintenance', 'has_telephone', 'foreign', 'risk']

# german_dataset = pd.read_csv(os.path.join(DATASET_PATH, "german_data.csv"), sep=' ', names=COLUMNS)

# class_feature = "risk"
# categorical_features = [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
# to_encode = [c for no, c in enumerate(COLUMNS) if no in categorical_features]

# d = defaultdict(preprocessing.LabelEncoder)

# german_dataset[to_encode] = german_dataset[to_encode].apply(lambda x: d[x.name].fit_transform(x))

# X = german_dataset.loc[:, german_dataset.columns!=class_feature]
# y = german_dataset[class_feature]

# Calculations

In [ ]:
from joblib import Parallel, delayed
from functools import partial

def calc_metrics(feature_range, cat, non_cat, X_primes, X):
    mean_hoem = np.mean([heterogeneous_euclidean_overlap_metric(X.values, X_prime.values, rfe.X_train_stats['range'],
                                                                  rfe.categorical_features,
                                                                  rfe.non_categorical_features) for _, X_prime in X_primes.iterrows()])
    
    d = diversity(X_primes, feature_range, cat, non_cat)
    return [mean_hoem, d]
   

CLASSES = y.unique()

SPLITS = 10

skf = StratifiedKFold(n_splits=SPLITS, shuffle=True, random_state=1000)
eps = 0.1
categorical_features = [1, 3, 5, 6, 7, 8, 9, 13]
frozen_features = [8, 9]
left_frozen_features = [0]

scores = defaultdict(list)

accuracy = []
split = 0

total_found = defaultdict(list)

for train_index, test_index in skf.split(X, y):
    start_time = time.time()
    print(split+1, "/", SPLITS)
    split += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf = RandomForestClassifier(n_estimators=100, max_depth=5)
    rf.fit(X_train, y_train)
    accuracy.append(accuracy_score(y_test, rf.predict(X_test)))
    
#     X_test = X_test.sample(100)
    
    y_hat = rf.predict(X_test)
    X_test_0 = X_test[y_hat==CLASSES[0]]
    X_test_1 = X_test[y_hat==CLASSES[1]]

    
    rfe = RandomForestExplainer(rf, X_train, y_train, categorical_features=categorical_features, 
                                        left_frozen_features=left_frozen_features, frozen_features=frozen_features)
    
    
    cfs0 = rfe.explain_with_multiple_metrics(X_test_0, CLASSES[1], eps=eps, k=5, metrics=('hoem', 'unmatched_components'), n_jobs=N_JOBS)
    cfs1 = rfe.explain_with_multiple_metrics(X_test_1, CLASSES[0], eps=eps, k=5, metrics=('hoem', 'unmatched_components'), n_jobs=N_JOBS)

    found_0 = sum([1 for i in range(len(cfs0)) if len(cfs0[i]) >= 2])
    found_1 = sum([1 for i in range(len(cfs1)) if len(cfs1[i]) >= 2])
    
    total_found['0'] += [found_0, len(cfs0)]
    total_found['1'] += [found_1, len(cfs1)]
    
    print(f"found_0: {found_0} / {X_test_0.shape[0]}")
    print(f"found_1: {found_1} / {X_test_1.shape[0]}")
    
    
    rfe = RandomForestExplainer(rf, X_train, y_train, categorical_features=categorical_features, 
                                        left_frozen_features=left_frozen_features, frozen_features=frozen_features)
    
    cfs0_single_hoem = rfe.explain_with_single_metric(X_test_0, CLASSES[1], eps=eps, metric='hoem', k=5, limit=None, n_jobs=N_JOBS)
    cfs1_single_hoem = rfe.explain_with_single_metric(X_test_1, CLASSES[0], eps=eps, metric='hoem', k=5, limit=None, n_jobs=N_JOBS)
    
    rfe = RandomForestExplainer(rf, X_train, y_train, categorical_features=categorical_features, 
                                        left_frozen_features=left_frozen_features, frozen_features=frozen_features)
    
    cfs0_single_ucr = rfe.explain_with_single_metric(X_test_0, CLASSES[1], eps=eps, metric='unmatched_components', k=5, limit=None, n_jobs=N_JOBS)
    cfs1_single_ucr = rfe.explain_with_single_metric(X_test_1, CLASSES[0], eps=eps, metric='unmatched_components', k=5, limit=None, n_jobs=N_JOBS)
    
#     cfs0_single_im = rfe.explain_with_single_metric(X_test_0, CLASSES[1], eps=eps, metric='implausibility_single', k=5, limit=None, n_jobs=N_JOBS)
#     cfs1_single_im = rfe.explain_with_single_metric(X_test_1, CLASSES[0], eps=eps, metric='implausibility_single', k=5, limit=None, n_jobs=N_JOBS)
    

    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs0[i], X_test_0.iloc[i]) for i in range(len(cfs0)) if len(cfs0[i]) >= 2)
    scores['multi0'].append(results)
    
    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs1[i], X_test_1.iloc[i]) for i in range(len(cfs1)) if len(cfs1[i]) >= 2)
    scores['multi1'].append(results)

    
    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs0_single_hoem[i].iloc[:len(cfs0[i])], X_test_0.iloc[i]) for i in range(len(cfs0)) if len(cfs0[i]) >= 2)
    scores['hoem0'].append(results)

    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs1_single_hoem[i].iloc[:len(cfs1[i])], X_test_1.iloc[i]) for i in range(len(cfs1)) if len(cfs1[i]) >= 2)
    scores['hoem1'].append(results)
    
    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs0_single_ucr[i].iloc[:len(cfs0[i])], X_test_0.iloc[i]) for i in range(len(cfs0)) if len(cfs0[i]) >= 2)
    scores['uc0'].append(results)
    
    results = Parallel(n_jobs=N_JOBS)(delayed(partial(calc_metrics, rfe.X_train_stats['range'],
                                                     rfe.categorical_features, rfe.non_categorical_features)
                                             )(cfs1_single_ucr[i].iloc[:len(cfs1[i])], X_test_1.iloc[i]) for i in range(len(cfs1)) if len(cfs1[i]) >= 2)
    scores['uc1'].append(results)
    

    print("Split time: ", time.time() - start_time, "s")


    
    

accuracy_mean, accuracy_std = np.mean(accuracy), np.std(accuracy)
accuracy_mean

In [ ]:
sum(total_found['0'][1::2])

In [ ]:
sum(total_found['1'][1::2])

In [ ]:
import itertools

scores2 = {}

for key, value in scores.items():
    score = np.array(list(itertools.chain.from_iterable(value)))
#     print(score)
    
    scores2[key] = score

scores2

In [ ]:
scores3 = {'multi': np.vstack([scores2['multi0'], scores2['multi1']]),
          'hoem': np.vstack([scores2['hoem0'], scores2['hoem1']]),
          'uc': np.vstack([scores2['uc0'], scores2['uc1']])
          }

In [ ]:
total_found_num = sum(total_found['0'][::2]) + sum(total_found['1'][::2])
total_found_denum = sum(total_found['0'][1::2]) + sum(total_found['1'][1::2])

total_found_num, total_found_denum

In [ ]:
# XLABELS = [f"pareto {CLASSES[0]} -> {CLASSES[1]}", f"pareto {CLASSES[0]} -> {CLASSES[1]}", f"hoem {CLASSES[0]} -> {CLASSES[1]}",
#            f"hoem {CLASSES[0]} -> {CLASSES[1]}", f"unmatched_components {CLASSES[0]} -> {CLASSES[1]}", f"unmatched_components {CLASSES[0]} -> {CLASSES[1]}"]

XLABELS = [f"pareto_optimal (hoem, u_c)", f"hoem", f"unmatched_components"]

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
y_pos = np.arange(len(scores3.keys()))
means = np.array([np.mean(s, axis=0) for s in scores3.values()])
stds = np.array([np.std(s, axis=0) for s in scores3.values()])

width=0.45


plt.barh(y_pos, means[:, 0], width, xerr=stds[:, 0], label='mean_hoem (lower better)')
plt.barh(y_pos+width, means[:, 1], width, xerr=stds[:, 1], label='diversity (higher better)')

ax.set_yticks(y_pos, labels=XLABELS)
ax.legend()
ax.grid()
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Value')
ax.set_ylabel("Metric")
ax.set_title(f"Adult dataset, 10-fold-cv. Random 100 samples from test data. Evaluation of set with counterfactuals")

plt.show()

In [ ]:
for no, metric in enumerate(XLABELS):
    print(f"{metric} & {means[no, 0]:1.3f}({stds[no, 0]:1.3f}) & {means[no, 1]:1.3f}({stds[no, 1]:1.3f}) \\\\")